In [368]:
import pandas as pd
import numpy as np
import math
from unicodedata import category
import sys

In [369]:
df=pd.read_csv("Train_B_Bayesian.csv")

In [370]:
def normal(x,mu,sigma):
    return (1/(math.sqrt(2*np.pi)*sigma))*np.exp(-(((x-mu)/sigma)**2)/2)

In [371]:
def naive_bayes(train,test):
    posframe=train[train['is_patient']==1]
    negframe=train.drop(posframe.index)
    P={}
    N={}
    for i in train.columns[:-2]:
        P[i]={}
        N[i]={}
        if i=='gender':
            P[i][0]=len(posframe[posframe[i]==0])/len(posframe)
            N[i][0]=len(negframe[negframe[i]==0])/len(negframe)
            P[i][1]=len(posframe[posframe[i]==1])/len(posframe)
            N[i][1]=len(negframe[negframe[i]==1])/len(negframe)
        else:
            P[i]['mean']=posframe[i].mean()
            P[i]['std']=posframe[i].std()
            N[i]['mean']=negframe[i].mean()
            N[i]['std']=negframe[i].std()
    p1=np.zeros(len(test))
    p2=np.zeros(len(test))
    for i in range(0,174):
        p_prob = 1
        n_prob = 1
        for j in test.columns[:-2]:
            if j=='gender':
                p_prob*=P[j][test[j][i]]
                n_prob*=N[j][test[j][i]]
            else:
                p_prob*=normal(test[j][i],P[j]['mean'],P[j]['std'])
                n_prob*=normal(test[j][i],N[j]['mean'],N[j]['std'])
            p1[i]=(p_prob*len(posframe))/len(train)
            p2[i]=(n_prob*len(posframe))/len(train)
    return (p1,p2)

In [372]:
S = df.isnull().sum()
S = S[S!=0]
for i in S.index:
    m = df[i].mean()
    for j in df.index:
        if(np.isnan(df[i][j])):
            df.loc[j,i]=m
            print(i,j,df[i][j])
MAX_OC=0
for j in df.index:
    df.loc[j,'OC']=0
for i in df.columns[:-2]:
    if(df[i].dtype=='object'):
        df[i] = df[i].astype('category').cat.codes
    m=df[i].mean()
    s=df[i].std()
    for j in df.index:
        if(df[i][j]>2*m+5*s):
            df.loc[j,'OC'] += 1
            if(df.loc[j,'OC']>MAX_OC):
                MAX_OC=df.loc[j,'OC']
#print(MAX_OC)
for j in df.index:
    if(df.loc[j,'OC']==MAX_OC):
        df=df.drop(j)

alkphos 209 0.9470639032815197
alkphos 241 0.9470639032815197
alkphos 253 0.9470639032815197
alkphos 312 0.9470639032815197


In [373]:
X=[]
train=df.sample(frac=0.7)
test=df.drop(train.index)
test=test.reset_index(drop=True)
T=np.array_split(train,5)
dflist=[]
dflist.append(pd.concat([T[1],T[2],T[3],T[4]]))
dflist.append(pd.concat([T[0],T[2],T[3],T[4]]))
dflist.append(pd.concat([T[0],T[1],T[3],T[4]]))
dflist.append(pd.concat([T[0],T[1],T[2],T[4]]))
dflist.append(pd.concat([T[0],T[1],T[2],T[3]]))

In [374]:
pred1=np.zeros(len(test))
pred2=np.zeros(len(test))
for i in range(0,5):
    (p1,p2)=naive_bayes(dflist[i],test)
    pred1=pred1+p1
    pred2=pred2+p2
pred1=pred1/5
pred2=pred2/5
pred=[]
match=0
for i in range(0,len(test)):
    if(pred1[i]>pred2[i]):
        pred.append(1)
    else:
        pred.append(2)
    if(pred[i]==test['is_patient'][i]):
        match=match+1
accuracy=match/len(test)
print("Final accuracy with 5-fold cross-validation:",accuracy)

Final accuracy with 5-fold cross-validation: 0.5919540229885057


In [375]:
def naive_bayes_laplace_correction(train,test,alpha):
    posframe=train[train['is_patient']==1]
    negframe=train.drop(posframe.index)
    P={}
    N={}
    for i in train.columns[:-2]:
        P[i]={}
        N[i]={}
        if i=='gender':
            P[i][0]=(len(posframe[posframe[i]==0])+alpha)/(len(posframe)+alpha*2)
            N[i][0]=(len(negframe[negframe[i]==0])+alpha)/(len(negframe)+alpha*2)
            P[i][1]=(len(posframe[posframe[i]==1])+alpha)/(len(posframe)+alpha*2)
            N[i][1]=(len(negframe[negframe[i]==1])+alpha)/(len(negframe)+alpha*2)
        else:
            P[i]['mean']=posframe[i].mean()
            P[i]['std']=posframe[i].std()
            N[i]['mean']=negframe[i].mean()
            N[i]['std']=negframe[i].std()
    p1=np.zeros(len(test))
    p2=np.zeros(len(test))
    for i in range(0,174):
        p_prob = 1
        n_prob = 1
        for j in test.columns[:-2]:
            if j=='gender':
                p_prob*=P[j][test[j][i]]
                n_prob*=N[j][test[j][i]]
            else:
                p_prob*=normal(test[j][i],P[j]['mean'],P[j]['std'])
                n_prob*=normal(test[j][i],N[j]['mean'],N[j]['std'])
            p1[i]=(p_prob*len(posframe))/len(train)
            p2[i]=(n_prob*len(posframe))/len(train)
    return (p1,p2)

In [376]:
p1=np.zeros(len(test))
p2=np.zeros(len(test))
alpha=1
(p1,p2)=naive_bayes_laplace_correction(train,test,alpha)
corr=0
prediction=[]
for i in range(0,len(test)):
    if(p1[i]>p2[i]):
        prediction.append(1)
    else:
        prediction.append(2)
    if(prediction[i]==test['is_patient'][i]):
        corr=corr+1
acc=corr/len(test)
print("Final accuracy with Laplace correction=",acc)

Final accuracy with Laplace correction= 0.5977011494252874
